# Reference used: 
- https://dzone.com/articles/algorithm-week-homomorphic

## Sub References: 
- https://www.cs.cmu.edu/~rwh/theses/okasaki.pdf
- https://pdos.csail.mit.edu/papers/otfvec/paper.pdf, 


## Properties of a Homomorphic Function: 

    k.H(m + n + o) = k.H(m) + k.H(n) + k.H(o) = H(k.m) + H(k.n) + H(k.o)

    The homomorphic hash function is:
        H(x) = g^x (mod q)

    here, sum = a + b + c + d + e -> sum is the key and
    a, b, ..., e are the subkeys that are generated from the key

In [162]:
import random
import math
import pickle

PASS_LEN = int(input("Enter maximum password length"))
BLOCK_LEN = int(input("Enter block size"))

In [163]:
def isPrime(x: int) -> bool:
    '''
    Checks if the given number is a prime
    '''
    for i in range(2, int(math.sqrt(x)) + 1):
        if(x % i == 0):
            return False

    return True

In [164]:
def primegen(start = 1001, step = 1) -> list:
    '''
    Generates a set of primes P to be used for for hashing
    '''
    i = start
    primes = []
    while (len(primes) < BLOCK_LEN):
        if(isPrime(i)):
            primes.append(i)
        i+=step
    # print(f'primes({start}, {step}): {primes}')

    return primes

In [165]:
def keygen(P = 1001) -> int:
    '''
    Generates a set of primes Q to be used for hashing
    Q follows the property: Q % P == 1
    '''
    keys = []
    i = 1
    # print(f"P: {P}\nQ:")
    while len(keys) < BLOCK_LEN:
        Q = P * i + 1
        if(isPrime(Q) and Q % P == 1):
            # print(f"\t{Q}")
            keys.append(Q)
        i+=1
    k = random.randint(0, BLOCK_LEN-1)
    # print(f'keygen({P}): {keys} - {keys[k]}')
    
    return keys[k]

In [166]:
def HomoHash(a: int, b: int, x: int, k = 1) -> int:
    '''
    The main homomorphic hash function: a^(k.x) (mod b)
    '''
    return pow(a, k*x, b)

In [167]:
def obfuscate(x: list, k = [1 for j in range(BLOCK_LEN)]) -> list:
    '''
    Obfuscates all the subkeys to be sent by multiplying them with a random secret number k
    '''
    obfuscated_message = [[0 for j in range(BLOCK_LEN)] for i in range(len(x))] 
    for i in range(len(x)):
        for j in range(BLOCK_LEN):
            obfuscated_message[i][j] = x[i][j]*k[j]
            # print(f"obfuscated_message[{i}][{j}] = x[{i}][{j}]*k[{j}] = {x[i][j]}*{k[j]} = {obfuscated_message[i][j]}")

    return obfuscated_message

In [168]:
def client_hash(x: list, g: list, q: list, p: list, k = [1 for j in range(BLOCK_LEN)])  -> list:
    '''
    implements "add": g^k(a+b) (mod q)
    # to compute the hash:
    ## compute the sum of the characters by adding each one
    ## hash that sum by raising g to the power of the sum (mod q)
    '''
    sums = [0 for j in range(BLOCK_LEN)]
    hashed_sums = [0 for j in range(BLOCK_LEN)]
    for i in range(BLOCK_LEN): # for i in range(0, PASS_LEN//BLOCK_LEN + 1): sum+= i%p
        # sum = 0 and replace hashed_sums with sum
        for j in range(len(x)):
            sums[i] += (x[j][i] % p[i])
        sums[i] %= p[i]
        hashed_sums[i] = HomoHash(g[i], q[i], k[i]*(sums[i]))
    # print(hashed_sums)
    
    return hashed_sums

In [169]:
def server_hash(x: list, g: list, q: list, p: list, k = [1 for j in range(BLOCK_LEN)])  -> tuple:
    '''
    implements "multiply": g^k.a * g^k.b (mod q)
    # hashed_products are the keys computed by each individual server
    # to compute the hash:
    ## compute the hash of each individual character by raising g to the power of the character (mod q)
    ## compute the "sum" of the hashes of the individual characters by multiplying them (mod q)
    '''
    hashed_products = [[1 for j in range(BLOCK_LEN)] for i in range(len(x))]
    products = [1 for j in range(BLOCK_LEN)]
    # for i in range(0, PASS_LEN//BLOCK_LEN + 1): product *= pow(g, q, x[i]%p)
    for i in range(BLOCK_LEN):
        for j in range(len(x)):
            temp = HomoHash(g[i], q[i], k[i]*x[j][i])
            hashed_products[j][i] = temp
            products[i] *= temp
            products[i] %= q[i]
    # print(products)
    # print(hashed_products)

    return products, hashed_products

In [170]:
# Variables used in the script
# get password from the user
inp = input(f"Enter a password with maximum {PASS_LEN} characters: ") # private
# 'qwertyuiop'  
inp += ' '*(PASS_LEN-len(inp))
# hash message
m = [[ord(j) for j in inp[i:i+BLOCK_LEN]] for i in range(0, PASS_LEN, BLOCK_LEN)] # private
S = [[0 for j in range(BLOCK_LEN)] for i in range(len(m))] # distributed

Enter a password with exactly 17 characters: somthign confusing for everyone that breaks things


In [171]:
# Constants used in the hash function
## Choose prime p - 257
p = primegen(1001) # public

In [172]:
## Choose q such that `q % p == 1` or `p | (q - 1)` - 257*6 + 1
q = [keygen(x) for x in p] # public

In [173]:
## a random number g
g = [random.randint(50, 100) for i in range(BLOCK_LEN)] # public

In [174]:
## generate fuzz factors k
k = [random.randint(15, 50) for i in range(BLOCK_LEN)] # private

In [175]:
# Calculate all the hashes from all parties
client_tmp = client_hash(m, g, q, p)
client_tmp_obf = client_hash(m, g, q, p, k)
server_tmp = server_hash(m, g, q, p)
server_tmp_obf = server_hash(m, g, q, p, k)

In [176]:
print("\nHash Function:\n\t H(x, a, b, k) = a^(k.x) (mod b)\n")
print("Data generated: ")
print("\n\t Input (inp): ")
print("\t", inp)
print("\n\t Input encoded into blocks (M): ")
print("\t", m)
print("\n\t Primes generated (P): ")
print("\t", p)
print("\n\t Dependent primes (Q[i] | Q[i] % P[i] == 1): ")
print("\t", q)
print("\n\t Bases (G): ")
print("\t", g)
print("\n\t Fuzz factors (K): ")
print("\t", k)
print("\n")

print("Clean data: ")
print("\nClient: ")
print("\n\tClient's Hash: ", client_tmp)
print("\nServers: ")
print("\n\tIndividual server computation: ")
print("\tServer's hash: ", server_tmp[1])
print("\n\tNetwork wide computation: ")
print("\tDistributed hash: ", server_tmp[0])
if(client_tmp == server_tmp[0]):
    print("\nChecking:\n\n\tHashes are equal!")
print("\n")

print("Obfuscated data: ")
print("\nClient: ")
print("\n\tClient's Hash: ", client_tmp_obf)
print("\nServers: ")
print("\n\tIndividual server computation: ")
print("\tServer's Hash: ", server_tmp_obf[1])
print("\n\tNetwork wide computation: ")
print("\tDistributed hash: ", server_tmp_obf[0])
if(client_tmp_obf == server_tmp_obf[0]):
    print("\nChecking:\n\n\tObfuscated hashes are equal!")
print("\n")


Hash Function:
	 H(x, a, b, k) = a^(k.x) (mod b)

Data generated: 

	 Input (inp): 
	 somthign confusing for everyone that breaks things

	 Input encoded into blocks (M): 
	 [[115, 111, 109, 116, 104], [105, 103, 110, 32, 99], [111, 110, 102, 117, 115], [105, 110, 103, 32, 102]]

	 Primes generated (P): 
	 [1009, 1013, 1019, 1021, 1031]

	 Dependent primes (Q[i] | Q[i] % P[i] == 1): 
	 [12109, 12157, 2039, 40841, 2063]

	 Bases (G): 
	 [84, 89, 94, 91, 80]

	 Fuzz factors (K): 
	 [26, 37, 43, 17, 20]


Clean data: 

Client: 

	Client's Hash:  [4126, 5877, 1405, 10291, 1266]

Servers: 

	Individual server computation: 
	Server's hash:  [[11825, 5516, 659, 39876, 1484], [7370, 10848, 776, 7029, 479], [6236, 11536, 1686, 34708, 1959], [7370, 11536, 1481, 7029, 623]]

	Network wide computation: 
	Distributed hash:  [4126, 5877, 1405, 10291, 1266]

Checking:

	Hashes are equal!


Obfuscated data: 

Client: 

	Client's Hash:  [8014, 11929, 381, 20655, 1623]

Servers: 

	Individual server co